# How to Aggregate and Plot AmpTools .fit Results
You now have a whole bunch of fit results, stored as `.fit` files, that you need to start plotting. Within this tutorial you will learn how to:
1. Aggregate these fit and data files into flattened `.csv` files
2. Load and plot the `.csv` fit results using python's pandas and matplotlib libraries   

## Environment
1. At the top right of the notebook your language / kernel is listed. Make sure `.venv (Python 3.9.18)` is selected. If the option does not appear, make sure to the virtual environment is active (see [**section 1.2** in the README](../README.md) for details).
    * If you can't find it, you may need to run the command `python -m ipykernel install --prefix=/path/to/alternative/location --name=.venv --display-name "Python (.venv)"`
2. Next we want to ensure that our GlueX environment is setup. Normally we could simply run `source setup_gluex.csh` if we were doing this in the terminal, but we'll need to go about it a special way to run this in the jupyter notebook:

In [ ]:
from pathlib import Path
import os
import subprocess

# first lets define the parent dir (this session)
PARENT_DIR = Path.cwd().resolve().parent
print(PARENT_DIR)

# We'll also need to set the data directory
DATA_DIR = "/work/halld/gluex_workshop_data/tutorial_2025/session2e"

import sys
sys.path.insert(0, PARENT_DIR) # add the session directory to the list of directories Python uses to look for modules

# run the source script (done here in csh, but bash could be done instead)
command = f"source {PARENT_DIR}/../setup_gluex.csh && env"
proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, executable='/bin/csh')
output, _ = proc.communicate()

# Parse the environment variables
env_vars = {}
for line in output.decode().splitlines():
    key, value = line.split('=', 1)
    env_vars[key] = value

# add the environment variables
os.environ.update(env_vars)

## File Aggregation
We will want to create the following 2 files in preparation for our analysis:
1. `data.csv`: constructed from the `.root` data files in each mass bin, containing the information for that bin
2. `best_fits.csv`: contains all the fit results across the entire mass range, made from the "best" of all the randomized fits in each bin

### Data
If we want to plot our fit results, we need to include the original data we are actually fitting to. This information is unfortunately not included in the `.fit` files, so we need to read it into a `data.csv` file using [convert_to_csv.py](../scripts/convert_to_csv.py). These python scripts use `argparse`, so we can conveniently see its abilities through its help message

In [ ]:
%run -i $PARENT_DIR/scripts/convert_to_csv.py -h

Great, now lets see what (sorted) files we are going to combine, and run the script

In [ ]:
%run -i $PARENT_DIR/scripts/convert_to_csv.py -i $DATA_DIR/mass*/*Amplitude.root -p
%run -i $PARENT_DIR/scripts/convert_to_csv.py -i $DATA_DIR/mass*/*Amplitude.root

You should now see a `data.csv` file here in the [analysis directory](./)

### Fit Results
Now we will be combining all our `.fit` results across the mass bins into a flattened `.csv` file to prepare them for analysis via python. This is achieved again through the [convert_to_csv.py](../scripts/convert_to_csv.py) script, which behind the scenes interacts with [extract_fit_results.cc](../scripts/extract_fit_results.cc) to load the AmpTools `FitResults` class and import the information we need.

In [ ]:
%run -i $PARENT_DIR/scripts/convert_to_csv.py -i $DATA_DIR/mass*/omegapi.fit -o best_fits.csv

Alongside the fit result `best_fits.csv` you'll also find 2 other csvs which contain the full covariance and correlation matrix of each fit result

## Preprocessing
To ensure that our data is ready for analysis, lets load in our files using pandas dataframes and check them for any potential issues. First, we can simply print out the heads (first 5 rows) to get a sense of the structure. Note if you are working on VS Code and installed the suggested [DataWrangler extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.datawrangler), then you can open these dataframes directly from this notebook for easy viewing

In [ ]:
import pandas as pd

# load files
df_fit = pd.read_csv("best_fits.csv")
df_data = pd.read_csv("data.csv")

df_fit.head()

In [ ]:
df_data.head()

Lets check that we have the same number of data files and fit files, and for any missing / NaN values:   

In [ ]:
print(df_fit.shape[0] == df_data.shape[0])
print(f"Number of null values in the fit results: {df_fit.isnull().sum().sum()}")
print(f"Number of null values in the data: {df_data.isnull().sum().sum()}")

Finally we'll want to wrap our phase differences to all be within a $2\pi$ range, and converted from radians to degrees $(^\circ)$. Lets wrap them within $(-\pi,\pi]$ using our helper functions in [utils](./utils.py).

In [ ]:
import utils
utils.wrap_phases(df_fit)

## Analysis
With our data prepared, lets move on to analysis. To avoid having to edit these settings for every single figure, lets change the global defaults

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams.update({
    'figure.figsize': (10, 8),
    'figure.dpi': 100,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'axes.labelsize': 16,
    'legend.fontsize': 16,
    'xtick.major.width': 2.0,
    'ytick.major.width': 2.0,
    'xtick.minor.width': 1.8,
    'ytick.minor.width': 1.8,
    'lines.markersize': 10,
    'grid.alpha': 0.8,
    'axes.grid': True
})
plt.minorticks_on()
plt.close() # suppress outputting an empty plot

We'll also define a few common parameters to all plots, and print out what coherent sums and phase differences we have available to us

In [ ]:
mass_bins = df_data["m_center"]
bin_width = (df_data["m_high"] - df_data["m_low"])[0] # all bin widths are equal, so just use the first one
coherent_sums = utils.get_coherent_sums(df_fit)
phase_differences = utils.get_phase_differences(df_fit)

for sum_type, sum_list in coherent_sums.items():
    print(f"{sum_type} -> {sum_list}")
for amp_pair, phase_column in phase_differences.items():
    print(f"{amp_pair} -> {phase_column}")

### Mass independent fit results
One of the simplest plots to make is of the data we fit to with the fit result's intensity in bins of mass, with the fit result decomposed into the coherent sum of our $J^P$ values. In this case we only have a $b_1$ contribution, and so only $1^+$ contributes

In [ ]:
# I prefer this colormap over the default
jp_colors = matplotlib.colormaps["Dark2"].colors

fig, ax = plt.subplots()

# plot data as black dots
ax.errorbar(
    x=mass_bins, y=df_data["events"], xerr=bin_width / 2.0 , yerr=df_data["events_err"], 
    fmt="k.",label="Signal MC",
)

# Plot Fit Result as a grey histogram with its error bars
ax.bar(
    x=mass_bins, height=df_fit["detected_events"], width=bin_width,        
    color="0.1", alpha=0.15, label="Fit Result",
)
ax.errorbar(
    x=mass_bins, y=df_fit["detected_events"], yerr=df_fit["detected_events_err"],
    fmt=",", color="0.1", alpha=0.2, markersize=0,
)

# plot 1+
ax.errorbar(
    x=mass_bins, y=df_fit["1p"], xerr=bin_width/2.0, yerr=df_fit["1p_err"], 
    label=utils.convert_amp_name("1p"), # converts amplitude / phase differences to be in J^P L_m^(e) format
    linestyle="", # want only markers, no lines\
    marker="1",
    color=jp_colors[2]
)

# plot 1-
ax.errorbar(
    x=mass_bins, y=df_fit["1m"], xerr=bin_width/2.0, yerr=df_fit["1m_err"], 
    label=utils.convert_amp_name("1m"), # converts amplitude / phase differences to be in J^P L_m^(e) format
    linestyle="", # want only markers, no lines\
    marker="s",
    color=jp_colors[3]
)

ax.set_xlabel(r"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")
ax.set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")
ax.set_ylim(bottom=0.0)
ax.legend()
plt.show()

How do the individual waves perform though? Lets view them in a grid of $m$-projections and $L$ angular momenta values, where each plot shares the reflectivity contributions. The reflectivities will be colored according to the unofficial official convention for $\color{red}natural~(\varepsilon=+1)$ and $\color{blue}unnatural~(\varepsilon=-1)$ exchange

In [ ]:
# define some dictionaries to convert characters <-> integers
char_to_int = {"m": -1, "0": 0, "p": +1, "S": 0, "P": 1, "D": 2}
int_to_char = {-1: "m", 0: "0", +1: "p"}
pm_dict = {"m": "-", "p": "+"}

# sort the JPL values by the order of S, P, D, F waves, and the m-projections
jpl_values = sorted(coherent_sums["JPL"], key=lambda JPL: char_to_int[JPL[-1]])
m_ints = sorted({char_to_int[JPmL[-2]] for JPmL in coherent_sums["JPmL"]})

fig, axs = plt.subplots(
    nrows=len(jpl_values),
    ncols=len(m_ints),
    sharex=True,    
    #sharey=True, # uncomment to compare relative intensities
    figsize=(15, 10),
)

# iterate through JPL (sorted like S, P, D, F wave) and sorted m-projections
for row, jpl in enumerate(jpl_values):
    for col, m in enumerate(m_ints):

        # force sci notation so large ticklabels don't overlap with neighboring plots
        axs[row,col].ticklabel_format(axis="y", style="sci", scilimits=(0,0))
        
        # recombine jpl and m to get string needed to access the column
        JPmL = f"{jpl[0:2]}{int_to_char[m]}{jpl[-1]}"

        # set the labels for the first rows and columns
        if row == 0:
            axs[row, col].set_title(f"m={m}", fontsize=18)
        if col == 0:
            J = JPmL[0]
            P = pm_dict[JPmL[1]]
            L = JPmL[-1]
            axs[row, col].set_ylabel(rf"${J}^{{{P}}}{L}$", fontsize=18)

        # plot the negative reflectivity contribution        
        neg_plot = axs[row, col].errorbar(
            x=mass_bins, y=df_fit[f"m{JPmL}"], xerr=bin_width/2.0, yerr=df_fit[f"m{JPmL}_err"],
            marker="v", linestyle="", markersize=8,
            color="blue", 
            alpha=0.5, # prevent overlap from cluttering the view
            label=r"$\varepsilon=-1$",
        )
        # plot the positive reflectivity contribution
        pos_plot = axs[row, col].errorbar(
            x=mass_bins, y=df_fit[f"p{JPmL}"], xerr=bin_width/2.0, yerr=df_fit[f"p{JPmL}_err"],
            marker="^", linestyle="", markersize=8,
            color="red",
            alpha=0.5,
            label=r"$\varepsilon=+1$",
        )

# reset limits to 0 for all plots
for ax in axs.reshape(-1):
    ax.set_ylim(bottom=0)

# figure cosmetics
fig.text(0.5, 0.04, r"$\omega\pi^0$ inv. mass (GeV)", ha="center", fontsize=20)
fig.text(
    0.04, 0.5, f"Events / {bin_width:.3f} GeV", 
    ha="center", va="center", 
    rotation="vertical", rotation_mode="anchor", fontsize=20,
)

# the pos/neg_plot variables get overwritten in the loop, so we're only passing one set of handles to the figure,
# which is okay since all plots have the same legend. If we didn't do this, every single plots redundant legend would be displayed
fig.legend(handles=[pos_plot, neg_plot], loc="upper right") # 
plt.show()
pass

What about the phase differences? Lets take a look at our two dominant amplitudes from separate $J^P$ values and plot their corresponding phase difference together

In [ ]:
import numpy as np

fig, axs = plt.subplots(2, 1, sharex=True, gridspec_kw={"wspace": 0.0, "hspace": 0.07}, height_ratios=[3, 1])

amp1, amp2 = "p1p0S", "p1mpP"

# plot the first amplitude
axs[0].errorbar(
    x=mass_bins, y=df_fit[amp1], xerr=bin_width/2.0, yerr=df_fit[f"{amp1}_err"],
    marker="o", linestyle="", color="green",
    label=utils.convert_amp_name(amp1),
)
# plot the second amplitude
axs[0].errorbar(
    x=mass_bins, y=df_fit[amp2], xerr=bin_width/2.0, yerr=df_fit[f"{amp2}_err"],
    marker="s", linestyle="", color="purple",
    label=utils.convert_amp_name(amp2),
)

# plot the phase difference. Since there is an inherent ambiguity in the sign of the phase difference within
# the model, we need to plot both signs to accommodate for every possible phase motion
phase_dif = phase_differences[(amp1, amp2)]
axs[1].errorbar(
    x=mass_bins, y=df_fit[phase_dif], xerr=bin_width/2.0, yerr=df_fit[f"{phase_dif}_err"].abs(), 
    marker=".", linestyle="", color="black"
)
axs[1].errorbar(
    x=mass_bins, y=-df_fit[phase_dif], xerr=bin_width/2.0, yerr=df_fit[f"{phase_dif}_err"].abs(), 
    marker=".", linestyle="", color="black"
)

# cosmetics
axs[0].set_ylim(bottom=0.0)
axs[0].set_ylabel(f"Events / {bin_width:.3f} GeV", loc="top")

axs[1].set_yticks(np.linspace(-180, 180, 5))  # force to be in pi intervals
axs[1].set_ylim([-180, 180])
axs[1].set_ylabel(r"Phase Diff. ($^{\circ}$)", loc="center")
axs[1].set_xlabel(r"$\omega\pi^0$ inv. mass $(GeV)$", loc="right")

axs[0].legend(loc="upper left")

plt.show()

### Correlation Matrix
To see how correlated our amplitude real/imaginary parts are, lets load in the matrix for each file and view the matrix as a heatmap

In [ ]:
import seaborn as sns

df_corr = pd.read_csv("best_fits_corr.csv")
files = df_corr["file"].unique()

def convert_parameter_name(full_parameter):
    """
    Convert the parameter name to a more readable format. Assumed to be in
    reaction::sum::JPml_re/im format (where re OR im can be used)
    """

    # drop the reaction name
    sum = full_parameter.split("::")[1]
    parameter = full_parameter.split("::")[-1].split("_")[0] # assumed to be in jpml format
    re_im_flag = full_parameter.split("::")[-1].split("_")[1] 

    # convert to eJPmL format
    e = "p" if "ImagPosSign" or "RealNegSign" in sum else "m"
    parameter = f"{e}{parameter[:-1]}{parameter[-1].upper()}"    

    # convert to J^P L_m^(e) format
    parameter = utils.convert_amp_name(parameter)
    
    if re_im_flag == "re":
        parameter = f"Re({parameter})"
    elif re_im_flag == "im":
        parameter = f"Im({parameter})"
    else:
        raise ValueError(f"Unknown re/im flag: {re_im_flag}")

    return parameter

for file in files:    
    df_corr_file = df_corr[df_corr["file"] == file].copy()
    df_corr_file = df_corr_file.drop(columns=["file"]) # remove the file column
    df_corr_file = df_corr_file.loc[~df_corr_file["parameter"].str.contains("Bkgd", na=False)]
    df_corr_file = df_corr_file.drop(columns=[col for col in df_corr_file.columns if "Bkgd" in col])

    # remove columns whose correlations are all zero
    df_corr_file = df_corr_file.loc[:, (df_corr_file != 0).any(axis=0)]

    # Convert parameter headers and the parameter column
    df_corr_file = df_corr_file.rename(columns=lambda col: convert_parameter_name(col) if col != "parameter" else col)
    df_corr_file["parameter"] = df_corr_file["parameter"].apply(convert_parameter_name)

    df_corr_file = df_corr_file.set_index("parameter")

    df = df.loc[~df["parameter"].str.contains("1m", na=False)]
    df = df.drop(columns=[c for c in df.columns if "1m" in c])
    
    plt.figure(figsize=(15, 15))
    sns.heatmap(df_corr_file.corr(), annot=True, fmt=".1f", cmap="coolwarm", cbar=True)
    plt.title(f"Correlation Matrix - {file}")    
    plt.tight_layout()
    plt.show()